# نحوه قالب‌بندی ورودی‌ها برای مدل‌های ChatGPT

این نوت‌بوک توضیح می‌دهد که چگونه می‌توان ورودی‌های متنی را برای مدل‌های ChatGPT قالب‌بندی کرد. این مدل‌ها شامل GPT-3.5 و GPT-4 هستند که از رابط چت‌محور استفاده می‌کنند.

به‌جای ارسال یک رشته‌ی ساده به عنوان ورودی، این مدل‌ها انتظار دارند **لیستی از پیام‌ها** دریافت کنند. هر پیام یک دیکشنری با دو فیلد ضروری است:
- `role` (که می‌تواند `system`، `user` یا `assistant` باشد)
- `content` (محتوای پیام)

مدل از این پیام‌ها به‌عنوان سابقه‌ی گفتگو استفاده می‌کند و خروجی آن، ادامه‌ی منطقی همان گفتگو خواهد بود.


In [6]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade openai


## ۱. کتابخانه openai را وارد کنید

In [7]:
# import the OpenAI Python library for calling the OpenAI API
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

## ۲. یک نمونه از فراخوانی API برای تولید پاسخ چت

پارامترهای مورد نیاز برای فراخوانی API تولید پاسخ چت:

**ضروری**
- `model`: نام مدلی که می‌خواهید استفاده کنید (مثلاً: `gpt-3.5-turbo`, `gpt-4`, `gpt-3.5-turbo-16k-1106`)
- `messages`: یک لیست از شیءهای پیام (message)، که هر کدام شامل دو فیلد ضروری هستند:
  - `role`: نقش فرستنده پیام (می‌تواند یکی از `system`، `user`، `assistant` یا `tool` باشد)
  - `content`: محتوای پیام (مثلاً: `برایم یک شعر زیبا بنویس`)

هر پیام می‌تواند به‌صورت اختیاری شامل فیلد `name` نیز باشد که به فرستنده پیام یک نام می‌دهد، مثلاً: `example-user`، `Alice`، یا `BlackbeardBot`. توجه داشته باشید که نام‌ها نباید شامل فاصله (space) باشند.

**اختیاری**
- `frequency_penalty`: بر اساس تکرار واژه‌ها، احتمال تکرار آن‌ها را کاهش می‌دهد.
- `logit_bias`: احتمال ظاهر شدن برخی توکن‌ها را با مقادیر بایاس تغییر می‌دهد.
- `logprobs`: در صورت فعال بودن، احتمال لگاریتمی توکن‌های خروجی را برمی‌گرداند.
- `top_logprobs`: تعداد توکن‌هایی که بیشترین احتمال را دارند در هر موقعیت مشخص می‌کند.
- `max_tokens`: حداکثر تعداد توکن‌های تولیدشده در پاسخ چت را تعیین می‌کند.
- `n`: تعداد پاسخ‌های جایگزین برای هر ورودی را مشخص می‌کند.
- `presence_penalty`: احتمال ظاهر شدن توکن‌هایی را که قبلاً در متن آمده‌اند کاهش می‌دهد.
- `response_format`: فرمت خروجی را مشخص می‌کند، مثلاً حالت JSON.
- `seed`: با مقداردهی اولیه ثابت، تولید پاسخ تکرارشونده (deterministic) را ممکن می‌سازد.
- `stop`: تا ۴ رشته را مشخص می‌کند که در صورت رسیدن به آن‌ها، مدل تولید پاسخ را متوقف کند.
- `stream`: خروجی را به صورت بخش‌بخش (streaming) و زنده برمی‌گرداند.
- `temperature`: میزان تصادفی بودن نمونه‌گیری را بین ۰ تا ۲ تعیین می‌کند.
- `top_p`: از نمونه‌گیری هسته‌ای (nucleus sampling) استفاده می‌کند و فقط توکن‌هایی را در نظر می‌گیرد که مجموع احتمالشان به `top_p` برسد.
- `tools`: لیستی از توابعی که مدل می‌تواند فراخوانی کند.
- `tool_choice`: نحوه انتخاب تابع توسط مدل را کنترل می‌کند (none/auto/function).
- `user`: شناسه منحصربه‌فرد کاربر نهایی، برای نظارت و جلوگیری از سوءاستفاده.

از ژانویه ۲۰۲۴، همچنین می‌توانید به صورت اختیاری لیستی از `functions` ارائه دهید که به GPT اجازه می‌دهد خروجی JSON تولید کند تا به عنوان ورودی یک تابع استفاده شود. برای جزئیات بیشتر، به [مستندات رسمی](https://platform.openai.com/docs/guides/function-calling)، [مرجع API](https://platform.openai.com/docs/api-reference/chat) یا راهنمای Cookbook [نحوه فراخوانی توابع با مدل‌های چت](How_to_call_functions_with_chat_models.ipynb) مراجعه کنید.

معمولاً یک مکالمه با یک پیام `system` آغاز می‌شود که نحوه رفتار مدل را تعیین می‌کند، سپس پیام‌های متناوب بین `user` و `assistant` ادامه پیدا می‌کند — اما پیروی از این ساختار الزامی نیست.

بیایید یک نمونه فراخوانی واقعی API را ببینیم تا ببینیم فرمت چت در عمل چگونه کار می‌کند.


In [8]:
# Example OpenAI Python library request
MODEL = "gpt-3.5-turbo"
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: <your Op*******************************var>. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
print(json.dumps(json.loads(response.model_dump_json()), indent=4))

البته، این هم توضیح فیلدهای شیء پاسخ به زبان فارسی:

* **`id`**: شناسه درخواست
* **`choices`**: لیستی از گزینه‌های تکمیل (معمولاً فقط یک گزینه است، مگر اینکه مقدار `n` بزرگتر از ۱ باشد)

  * **`finish_reason`**: دلیل توقف مدل در تولید متن (مثلاً `stop` برای پایان طبیعی، یا `length` اگر محدودیت `max_tokens` رسید)
  * **`index`**: شماره اندیس گزینه در لیست گزینه‌ها
  * **`logprobs`**: اطلاعات احتمال لگاریتمی برای گزینه انتخاب شده
  * **`message`**: شیء پیام تولید شده توسط مدل

    * **`content`**: متن پیام
    * **`role`**: نقش نویسنده پیام (مثلاً assistant یا user)
    * **`tool_calls`**: تماس‌هایی که مدل با ابزارها زده، مانند فراخوانی توابع (اگر ابزار داده شده باشد)
* **`created`**: زمان ایجاد درخواست (تایم‌استمپ)
* **`model`**: نام کامل مدل استفاده شده برای تولید پاسخ
* **`object`**: نوع شیء برگشتی (مثل `chat.completion`)
* **`system_fingerprint`**: یک شناسه خاص که نشان‌دهنده تنظیمات بک‌اند است که مدل با آن اجرا شده
* **`usage`**: تعداد توکن‌های مصرف شده برای تولید پاسخ، شامل توکن‌های ورودی (prompt)، خروجی (completion) و مجموع آنها

اگر نیاز داشتی بیشتر توضیح بدم یا نمونه کد هم بخوای، بگو.


فقط پاسخ را با این دستور استخراج کنید:

In [ ]:
response.choices[0].message.content


حتما! منظور این است که حتی کارهایی که مکالمه‌ای نیستند هم می‌توانند با قالب چت انجام شوند، به این شکل که دستور یا درخواست اصلی را در اولین پیام کاربر قرار می‌دهیم.

مثلاً اگر بخواهیم از مدل بخواهیم برنامه‌نویسی غیرهمزمان (asynchronous programming) را به سبک دزد دریایی معروف «بلک‌برد» (Blackbeard) توضیح دهد، مکالمه را اینطور تنظیم می‌کنیم:

---

**User:**
"لطفاً برنامه‌نویسی غیرهمزمان را به سبک دزد دریایی بلک‌برد توضیح بده."

---

مدل بر اساس این پیام، جواب را به شکل یک مکالمه تولید می‌کند اما لحن و سبک آن مطابق درخواست کاربر خواهد بود.

این روش به شما امکان می‌دهد هر نوع درخواست و وظیفه‌ای، چه مکالمه‌ای و چه غیرمکالمه‌ای، را در قالب یک گفت‌وگو با مدل بیان کنید.
اگر بخواهی، می‌توانم یک مثال کامل JSON یا کد درخواست API هم برات بفرستم.


In [ ]:
# example with a system message
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain asynchronous programming in the style of the pirate Blackbeard."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)


In [ ]:
# example without a system message
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Explain asynchronous programming in the style of the pirate Blackbeard."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)


## ۳. نکات راهنمایی دادن به مدل gpt-3.5-turbo-0301
روش‌های بهینه برای راهنمایی مدل‌ها ممکن است بین نسخه‌های مختلف مدل تغییر کند. نکاتی که در ادامه آمده برای مدل gpt-3.5-turbo-0301 کاربرد دارد و ممکن است برای مدل‌های آینده صادق نباشد.`

### پیام‌های سیستم
پیام سیستم می‌تواند برای آماده‌سازی (priming) دستیار با شخصیت‌ها یا رفتارهای متفاوت استفاده شود.

توجه داشته باشید که مدل gpt-3.5-turbo-0301 معمولاً به پیام سیستم به اندازه مدل‌های gpt-4-0314 یا gpt-3.5-turbo-0613 توجه نمی‌کند.
به همین دلیل، برای gpt-3.5-turbo-0301 توصیه می‌شود که دستورالعمل‌های مهم را در پیام کاربر (user message) قرار دهید.

برخی توسعه‌دهندگان با این روش موفق بوده‌اند که پیام سیستم را در طول مکالمه به سمت پایان منتقل کنند تا توجه مدل هنگام طولانی‌تر شدن گفتگو کمتر منحرف شود.

In [ ]:
# An example of a system message that primes the assistant to explain concepts in great depth
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a friendly and helpful teaching assistant. You explain concepts in great depth using simple terms, and you give examples to help people learn. At the end of each explanation, you ask a question to check for understanding"},
        {"role": "user", "content": "Can you explain how fractions work?"},
    ],
    temperature=0,
)

print(response.choices[0].message.content)


In [ ]:
# An example of a system message that primes the assistant to give brief, to-the-point answers
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a laconic assistant. You reply with brief, to-the-point answers with no elaboration."},
        {"role": "user", "content": "Can you explain how fractions work?"},
    ],
    temperature=0,
)

print(response.choices[0].message.content)


آموزش با چند مثال (Few-shot prompting)
در بعضی موارد، به جای گفتن مستقیم به مدل آنچه می‌خواهید، راحت‌تر است که با نشان دادن مثال‌هایی به مدل بفهمانید چه انتظاری دارید.

یکی از روش‌های نشان دادن خواسته به مدل، استفاده از پیام‌های نمونه جعلی (faked example messages) است.

برای مثال:

In [ ]:
# An example of a faked few-shot conversation to prime the model into translating business jargon to simpler speech
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)


برای اینکه روشن شود پیام‌های نمونه بخشی از یک گفت‌وگوی واقعی نیستند و مدل نباید به آنها ارجاع دهد، می‌توانید فیلد name در پیام‌های system را به example_user و example_assistant تغییر دهید.

با این کار، نمونه چندمثالی (few-shot example) بالا را می‌توان این‌گونه نوشت:

In [ ]:
# The business jargon translation example, but with example names for the example messages
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English."},
        {"role": "system", "name":"example_user", "content": "New synergies will help drive top-line growth."},
        {"role": "system", "name": "example_assistant", "content": "Things working well together will increase revenue."},
        {"role": "system", "name":"example_user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "system", "name": "example_assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)


هر بار که تلاش می‌کنید مکالمه‌ها را مهندسی کنید، همیشه در ابتدا موفق نخواهید بود.

اگر تلاش‌های اول شما جواب نداد، نترسید و روش‌های مختلف آماده‌سازی (priming) یا شرطی‌سازی (conditioning) مدل را آزمایش کنید.

مثلاً یک توسعه‌دهنده متوجه شد که وقتی پیامی از طرف کاربر به مدل می‌دهد با این مضمون:
«کار خیلی خوبی تا الان انجام دادی، این‌ها عالی بودند»
دقت پاسخ‌های مدل بهتر می‌شود و کیفیت بالاتری ارائه می‌کند.

برای ایده‌های بیشتر در مورد چگونگی افزایش قابلیت اطمینان مدل‌ها، می‌توانید راهنمای ما درباره روش‌هایی برای افزایش قابلیت اطمینان را مطالعه کنید. این راهنما برای مدل‌های غیرمکالمه‌ای نوشته شده اما بسیاری از اصول آن همچنان کاربرد دارند.

## ۴. شمارش توکن‌ها
وقتی درخواست خود را ارسال می‌کنید، API پیام‌ها را به دنباله‌ای از توکن‌ها تبدیل می‌کند.

تعداد توکن‌های استفاده‌شده روی موارد زیر تاثیر دارد:

هزینه درخواست

مدت زمانی که برای تولید پاسخ صرف می‌شود

زمان قطع شدن پاسخ در صورت رسیدن به حداکثر محدودیت توکن (۴۰۹۶ توکن برای gpt-3.5-turbo و ۸۱۹۲ توکن برای gpt-4)

شما می‌توانید از تابع زیر برای شمارش تعداد توکن‌هایی که یک لیست پیام مصرف می‌کند استفاده کنید.

توجه داشته باشید که روش دقیق شمارش توکن‌ها در مدل‌های مختلف ممکن است متفاوت باشد. پس نتایج تابع زیر تخمینی است و تضمینی دائمی نیست.

به‌خصوص، درخواست‌هایی که از ورودی‌های توابع اختیاری (optional functions input) استفاده می‌کنند، توکن‌های بیشتری نسبت به تخمین‌های پایین مصرف خواهند کرد.

برای مطالعه بیشتر درباره شمارش توکن‌ها، به چگونه با tiktoken توکن‌ها را بشماریم مراجعه کنید.

In [ ]:
import tiktoken


def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens


In [ ]:
# let's verify the function above matches the OpenAI API response
example_messages = [
    {
        "role": "system",
        "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "New synergies will help drive top-line growth.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Things working well together will increase revenue.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Let's talk later when we're less busy about how to do better.",
    },
    {
        "role": "user",
        "content": "This late pivot means we don't have time to boil the ocean for the client deliverable.",
    },
]

for model in [
    # "gpt-3.5-turbo-0301",
    # "gpt-4-0314",
    # "gpt-4-0613",
    "gpt-3.5-turbo-1106",
    "gpt-3.5-turbo",
    "gpt-4",
    "gpt-4-1106-preview",
    ]:
    print(model)
    # example token count from the function defined above
    print(f"{num_tokens_from_messages(example_messages, model)} prompt tokens counted by num_tokens_from_messages().")
    # example token count from the OpenAI API
    response = client.chat.completions.create(model=model,
    messages=example_messages,
    temperature=0,
    max_tokens=1)
    token = response.usage.prompt_tokens
    print(f'{token} prompt tokens counted by the OpenAI API.')
    print()
